# Table 3. Model performance

In [1]:
# function for model evaluation
F_model_evals = function(tmp) # tmp is a data.frame containing two columme: obs and pre
                 {
                  d_lm = with(tmp,lm(pre ~ obs))
                  model_evals = c(summary(d_lm)$r.squared,                              # R2
                                  d_lm$coefficients[2],                                 # Slope 
                                  d_lm$coefficients[1],                                 # intercept
                                  (sum((tmp$obs - tmp$pre)^2)/dim(tmp)[1])^0.5,         # RMSE
                                  dim(tmp)[1]                                           # N
                                  )                                            
                  }

In [2]:
nsite = c('how', 'das', 'dry', 'asm', 'dap','stp')
lens = length(nsite)+3
mevals <- data.frame(r.squared=rep(NA,lens),  slp=rep(NA,lens), itcp=rep(NA,lens), 
                     rmse = rep(NA,lens),  num=rep(NA,lens))
rownames(mevals) = c(nsite, 'all', 'savanna', 'pasture')

## load simulation results of PML, SW and TS models

In [3]:
workdir = 'D:/Field/NATT/modeling_et_afm_v16d/' # this maybe need change according where you place the directory of modeling_et

# Load data
fdata = read.csv(paste0(workdir, 'output/simulation_results.csv'))

# Load DIs data
fdata_di = read.csv(paste0(workdir, 'output/simulation_results_DIs.csv'))

In [4]:
# combine predicted et from the three models
edata = cbind(fdata[,1:2], # 1 for site and 2 for date
              fe_qcflag = fdata$fe_qcflag,   # latent heat flux quality flag
              ebr       = fdata$ebr,         # energy balance ratio
              et_ob     = fdata$et_ob,      # the observed ET
              et_pml    = fdata$et_pml,      # total ET predicted by PML model
              et_sw     = fdata$et_sw,       # total ET predicted by SW model
              et_ts     = fdata$et_ts,       # total ET predicted by TS model
              et_pml_di = fdata_di$et_pml_di,      # total ET predicted by PML-DI model
              et_sw_di  = fdata_di$et_sw_di,       # total ET predicted by SW-DI model
              et_ts_di  = fdata_di$et_ts_di)      # total ET predicted by TS-DI model)                

In [5]:
# data quality control
# remove stp years 2009 and 2015 when soil water content observation is abnormal.
edata$year = as.numeric(strftime(as.Date(edata$date), format='%Y'))
edata = edata[edata$site != 'stp' | edata$year !=2009,]
edata = edata[edata$site != 'stp' | edata$year !=2015,]

In [6]:
# remove NA records
edata = na.omit(edata)
# remove gap-filling percent >0.2
edata = subset(edata, fe_qcflag<0.2 & et_ob >0 & ebr > 0.7 & ebr < 1.3)

# evaluate model performance

In [7]:
## (1) evaluate model performance of pml model
# PML 
mevals1 = mevals
# each site
for (i in unique(nsite))
{ adata = edata[edata$site == i, ]
  tmp = data.frame(obs = adata$et_ob,
                   pre = adata$et_pml)
  mevals1[i,] = F_model_evals(tmp)
}
# all site
  tmp = data.frame(obs = edata$et_ob,
                   pre = edata$et_pml)
  mevals1[7,] = F_model_evals(tmp)
# all savanna sites
  data_sub = subset(edata, site == 'asm' | site == 'das' | site == 'dry' | site == 'how')
  tmp = data.frame(obs = data_sub$et_ob,
                   pre = data_sub$et_pml)
  mevals1[8,] = F_model_evals(tmp)
# group: pasture sites
  data_sub = subset(edata, site == 'dap' | site == 'stp')
  tmp = data.frame(obs = data_sub$et_ob,
                   pre = data_sub$et_pml)
  mevals1[9,] = F_model_evals(tmp)

In [8]:
# SW
mevals2 = mevals
# each site
for (i in unique(nsite))
{ adata = edata[edata$site == i, ]
  tmp = data.frame(obs = adata$et_ob,
                   pre = adata$et_sw)
  mevals2[i,] = F_model_evals(tmp)
}
# all site
  tmp = data.frame(obs = edata$et_ob,
                   pre = edata$et_sw)
  mevals2[7,] = F_model_evals(tmp)
# all savanna sites
  data_sub = subset(edata, site == 'asm' | site == 'das' | site == 'dry' | site == 'how')
  tmp = data.frame(obs = data_sub$et_ob,
                   pre = data_sub$et_sw)
  mevals2[8,] = F_model_evals(tmp)
# group: pasture sites
  data_sub = subset(edata, site == 'dap' | site == 'stp')
  tmp = data.frame(obs = data_sub$et_ob,
                   pre = data_sub$et_sw)
  mevals2[9,] = F_model_evals(tmp)

In [9]:
# TS
mevals3 = mevals
# each site
for (i in unique(nsite[1:4]))
{ adata = edata[edata$site == i, ]
  tmp = data.frame(obs = adata$et_ob,
                   pre = adata$et_ts)
  mevals3[i,] = F_model_evals(tmp)
}
# all site
  tmp = data.frame(obs = edata$et_ob,
                   pre = edata$et_ts)
  mevals3[7,] = F_model_evals(tmp)
# all savanna sites
  data_sub = subset(edata, site == 'asm' | site == 'das' | site == 'dry' | site == 'how')
  tmp = data.frame(obs = data_sub$et_ob,
                   pre = data_sub$et_ts)
  mevals3[8,] = F_model_evals(tmp)
# group: pasture sites
  data_sub = subset(edata, site == 'dap' | site == 'stp')
  tmp = data.frame(obs = data_sub$et_ob,
                   pre = data_sub$et_ts)
  mevals3[9,] = F_model_evals(tmp)

In [10]:
# PML-DI 
mevals4 = mevals
# each site
for (i in unique(nsite))
{ adata = edata[edata$site == i, ]
  tmp = data.frame(obs = adata$et_ob,
                   pre = adata$et_pml_di)
  mevals4[i,] = F_model_evals(tmp)
}
# all site
  tmp = data.frame(obs = edata$et_ob,
                   pre = edata$et_pml_di)
  mevals4[7,] = F_model_evals(tmp)
# all savanna sites
  data_sub = subset(edata, site == 'asm' | site == 'das' | site == 'dry' | site == 'how')
  tmp = data.frame(obs = data_sub$et_ob,
                   pre = data_sub$et_pml_di)
  mevals4[8,] = F_model_evals(tmp)
# group: pasture sites
  data_sub = subset(edata, site == 'dap' | site == 'stp')
  tmp = data.frame(obs = data_sub$et_ob,
                   pre = data_sub$et_pml_di)
  mevals4[9,] = F_model_evals(tmp)

In [11]:
# SW-DI
mevals5 = mevals
# each site
for (i in unique(nsite))
{ adata = edata[edata$site == i, ]
  tmp = data.frame(obs = adata$et_ob,
                   pre = adata$et_sw_di)
  mevals5[i,] = F_model_evals(tmp)
}
# all site
  tmp = data.frame(obs = edata$et_ob,
                   pre = edata$et_sw_di)
  mevals5[7,] = F_model_evals(tmp)
# all savanna sites
  data_sub = subset(edata, site == 'asm' | site == 'das' | site == 'dry' | site == 'how')
  tmp = data.frame(obs = data_sub$et_ob,
                   pre = data_sub$et_sw_di)
  mevals5[8,] = F_model_evals(tmp)
# group: pasture sites
  data_sub = subset(edata, site == 'dap' | site == 'stp')
  tmp = data.frame(obs = data_sub$et_ob,
                   pre = data_sub$et_sw_di)
  mevals5[9,] = F_model_evals(tmp)

In [12]:
# TS-DI
mevals6 = mevals
# each site
for (i in unique(nsite[1:4]))
{ adata = edata[edata$site == i, ]
  tmp = data.frame(obs = adata$et_ob,
                   pre = adata$et_ts_di)
  mevals6[i,] = F_model_evals(tmp)
}
# all site
  tmp = data.frame(obs = edata$et_ob,
                   pre = edata$et_ts_di)
  mevals6[7,] = F_model_evals(tmp)
# all savanna sites
  data_sub = subset(edata, site == 'asm' | site == 'das' | site == 'dry' | site == 'how')
  tmp = data.frame(obs = data_sub$et_ob,
                   pre = data_sub$et_ts_di)
  mevals6[8,] = F_model_evals(tmp)
# group: pasture sites
  data_sub = subset(edata, site == 'dap' | site == 'stp')
  tmp = data.frame(obs = data_sub$et_ob,
                   pre = data_sub$et_ts_di)
  mevals6[9,] = F_model_evals(tmp)

In [13]:
mevalscom <- cbind(mevals2, mevals1, mevals3, mevals5, mevals4,  mevals6)
#rownames(mevalscom) = c(nsite, 'allsite', 'savannas', 'pasture_grassland')
mevalscom

,r.squared,slp,itcp,rmse,num,r.squared.1,slp.1,itcp.1,rmse.1,num.1,...,r.squared,slp,itcp,rmse,num,r.squared.1,slp.1,itcp.1,rmse.1,num.1
how,0.7958577,0.8162904,0.5879543,0.5615227,3669,0.8013075,0.8668267,0.3584117,0.5637301,3669,...,0.8214373,0.9322713,0.207521916,0.5459282,3669,0.8275733,0.9027861,0.30384886,0.5254153,3669
das,0.7410770,0.8265860,0.3480796,0.5696898,2595,0.7637240,0.8482716,0.3232269,0.5428238,2595,...,0.8155879,0.9618133,0.071877018,0.5021744,2595,0.8020961,0.9638262,0.06391882,0.5254353,2595
dry,0.6228074,0.6662150,0.7150231,0.7321845,1796,0.6408714,0.7346023,0.5935178,0.7262077,1796,...,0.7977947,0.9924435,-0.003532951,0.5926179,1796,0.7851930,0.9515188,0.08674892,0.5928273,1796
asm,0.7165555,0.6369680,0.3777883,0.5080478,1451,0.7510417,0.6751493,0.3397468,0.4754043,1451,...,0.8203861,0.7864957,0.259026932,0.4032965,1451,0.7906977,0.7591729,0.27489901,0.4320308,1451
dap,0.8621002,0.8173110,0.4189516,0.6759577,1316,0.8765473,0.8526618,0.3102115,0.6331734,1316,...,0.8846101,0.8864531,0.195435127,0.6103214,1316,NA,NA,NA,NA,NA
stp,0.7810930,0.8336092,0.3208188,0.6238817,1856,0.8747260,0.9452160,0.2468923,0.5035527,1856,...,0.8681986,0.8725030,0.233204917,0.4786355,1856,NA,NA,NA,NA,NA
all,0.8319105,0.8359424,0.3846102,0.6061605,12683,0.8484780,0.8629680,0.3200063,0.5757276,12683,...,0.8756013,0.9287600,0.163474069,0.5275692,12683,0.8625509,0.9120334,0.21184817,0.5534657,12683
savanna,0.8225526,0.8343019,0.4017120,0.5922792,9511,0.8299718,0.8543754,0.3359458,0.5805638,9511,...,0.8677231,0.9455522,0.132763255,0.5243172,9511,0.8648605,0.9306023,0.16913400,0.5259861,9511
pasture,0.8291108,0.8279928,0.3576646,0.6459968,3172,0.8744788,0.8878735,0.2922700,0.5609769,3172,...,0.8801966,0.8796917,0.217001870,0.5372019,3172,0.8384082,0.8656845,0.28452041,0.6287022,3172


In [14]:
write.csv(mevalscom, file= 'table_3_model_evaluation_DIs.csv')